In [ ]:
# !python3 -m spacy download en_core_web_lg

In [1]:
!ls ../data/ADE20K-pairs/captions/ade20k_train_captions.jsonl

../data/ADE20K-pairs/captions/ade20k_train_captions.jsonl


In [2]:
import joblib, numpy as np

In [3]:
from pprint import pprint

In [4]:
import torch

In [5]:
torch.cuda.empty_cache()

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [7]:
if torch.cuda.is_available():
   device = torch.device("cuda:1")
   print("Running on the GPU")
else:
   device = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [8]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap").to(device)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [9]:
%%time
def get_question(answer, context, max_length=32):
    input_text = "answer: %s  context: %s </s>" % (answer, context)
    features = tokenizer([input_text], return_tensors='pt').to(device)
    output = model.generate(input_ids=features['input_ids'], 
                            attention_mask=features['attention_mask'],
                            num_beams=10,
                            no_repeat_ngram_size=2,
                            num_return_sequences=1,
                            early_stopping=True,
                            max_length=max_length)
    questions = [tokenizer.decode(o) for o in output]
    return questions[0].strip("<pad> question: ").strip("</s>")

context = "This picture describes about inside view of a hall, in this we can find group \
of people, few are sitting on the chairs, in the background we can see lights."
answer = "not many"


get_question(answer, context)

# output: question: Who created the RuPERTa-base

CPU times: user 2.11 s, sys: 269 ms, total: 2.38 s
Wall time: 2.38 s


'How many people are sitting on chairs in the picture?'

In [57]:
# context = 'In this image there are cushions on the bed. \
# Below it there are baskets. Background there is a wall having windows. \
# From the windows few trees are visible. Left side there is a blue curtain.'
# answer="blue"

# get_question(answer, context)


In [25]:
# c = 'there is a shelf'
# a = 'yes'

# get_question(a, c)

In [29]:
# context = "This picture describes about inside view of a hall, in this we can find group of people,\
# few are sitting on the chairs, in the background we can see lights."
# answer = "chairs"

# get_question(answer, context)

In [30]:
# context = "In the background I can see few curtains, a door, few candle stands, a fireplace, two lamps"
# answer = "curtains"

# get_question(answer, context)

In [10]:
context =  'In this picture I can see 3 stools and the countertops on the floor in front.\
On a countertop, I can see few things. \
In the background, I can see the wall and a cupboard. \
On the top of this picture, I can see the lights on the ceiling. \
On the left side of this picture, I ca can see a white color thing.'
# answer = "left"

# get_question(answer, context)

In [13]:
# !pip install -U spacy 

In [12]:
# !pip install -U 'spacy[cuda101]'

In [6]:
# !pip install git+https://github.com/LIAAD/yake

In [3]:
# !python3 -m pip install textacy

In [13]:
# !python3 -m spacy download en_core_web_lg

In [13]:
# !pip install -U spacy

In [173]:
import spacy
spacy.require_gpu(gpu_id=1)

nlp = spacy.load("en_core_web_lg")

In [12]:
%%time
#Question types: yes/no, what, where, how many, 

ignore_list = ['side','background','foreground','picture','image','color','thing','things','group',\
               'view','few','many','some','top','bottom','inside','outside','other','others',
               'object','objects','place'
              ]


def get_qa_pairs(text):
    doc = nlp(text)
    pos_answers = []
    pos_questions = []
    pairs = []

#     print(doc,"\n")
    for word in doc:
        if word.tag_ in ['NN','JJ','NNS','CD'] \
        and word.text.lower() not in ignore_list:
            pos_answers.append(word.lemma_)
    
    pos_answers=set(pos_answers) #remove duplicate answers
    
    for a in pos_answers:
        q = get_question(a, text)
        if q not in pos_questions: #remove duplicate questions
            pairs.append((q,a))
            pos_questions.append(q)
        
    return pairs
get_qa_pairs(context)
    

CPU times: user 1.86 s, sys: 117 ms, total: 1.97 s
Wall time: 8.36 s


[('In this picture few people are sitting on what?', 'chair'),
 ('What can we see in the background of the picture?', 'light'),
 ('What is the inside view of?', 'hall'),
 ('In this picture we can find a group of what?', 'people')]

In [39]:
# ## answers

# doc = nlp(context)
# pos_answers = []

# print(doc,"\n")
# for word in doc:
#     if word.tag_ in ['NN','JJ','NNS'] \
#     and word.text.lower() not in ['side','background','picture','image','color','thing']:
#         pos_answers.append(word.text)
#         print(word.text, word.tag_, word.dep_, ", Explanation:", spacy.explain(word.tag_))

In [103]:
# dependency_answers = []
# print(doc,"\n")
# for word in doc:
#     if word.tag_ in ['NN','JJ','NNS']: 
#         if word.dep_ in ['nsubj','obj','pobj','dobj']:
#             dependency_answers.append(word.text)
#             print(word.text, word.dep_, ", Explanation:", spacy.explain(word.dep_))


In [40]:
# pos_questions = []
# for i in pos_answers:
#     pos_questions.append(get_question(i, context))
# for i, j in zip(pos_questions, pos_answers):
#     pprint(i)
#     pprint(j)

### Adding questions thru captions

In [193]:
!ls

LICENSE      data_collator.py  prepare_data.py		  utils.py
README.md    eval.py	       question_generation.ipynb
__pycache__  notebooks	       run_qg.py
data	     pipelines.py      trainer.py


In [197]:
train_dir = "../../data/ADE20K-pairs/captions/ade20k_train_captions.jsonl"
val_dir = "../../data/ADE20K-pairs/captions/ade20k_validation_captions.jsonl"

In [198]:
import json

In [199]:
train_caps = dict()
val_caps = dict()


with open(train_dir, 'r') as f:
    lines = f.readlines()
    lines = lines
    for line in lines:
        load = json.loads(line)
        train_caps[load['image_id']] = load['caption']
        
with open(val_dir, 'r') as f:
    lines = f.readlines()
    for line in lines:
        load = json.loads(line)
        val_caps[load['image_id']] = load['caption']
        
len(train_caps.keys()), len(val_caps.keys())



(20210, 2000)

In [200]:
# train_qa = dict()
# c = 0
# for img_id, caption in train_caps.items():
#     qa_pairs = get_qa_pairs(caption)
#     train_qa[img_id] = qa_pairs
#     if not c%50:
#         print(c)
#         print(caption)
#         print(qa_pairs)
#     c+=1

    
# val_qa = dict()
# c = 0
# for img_id, caption in val_caps.items():
#     qa_pairs = get_qa_pairs(caption)
#     val_qa[img_id] = qa_pairs
#     if not c%50:
#         print(c)
#         print(caption)
#         print(qa_pairs)
#     c+=1
# train_qa

In [203]:
%cd ../../data/ADE20K-QA/

/project/dataset-exploration/clip-finetune-ade20k/data/ADE20K-QA


In [204]:
import jsonlines

In [205]:
# with jsonlines.open('qa_15000-18000.jsonl','w') as f:
#     for k, v in train_qa.items():
#         f.write({k:v})

In [206]:
train_qa = dict()
val_qa = dict()

with jsonlines.open('train_qa.jsonl','r') as f:
    for row in f:
        train_qa.update(row)        

with jsonlines.open('val_qa.jsonl','r') as f:
    for row in f:
        val_qa.update(row)        
len(train_qa.keys()), len(val_qa.keys())



(20210, 2000)

In [208]:
np.max([int(key.strip("ADE_train_")) for key in train_qa.keys()])

20210

In [209]:
set(val_qa.keys()).intersection(set(train_qa.keys()))

set()

In [210]:
!ls ../ADE20K-pairs/imgs/training

cultural       nature_landscape     transportation  work_place
home_or_hotel  shopping_and_dining  unclassified
industrial     sports_and_leisure   urban


In [211]:
import glob

train_img_dir = "../ADE20K-pairs/imgs/training/*/*/*.json"
val_img_dir = "../ADE20K-pairs/imgs/validation/*/*/*.json"

train_img_paths = glob.glob(train_img_dir)
val_img_paths = glob.glob(val_img_dir)

len(train_img_paths), len(val_img_paths)

(25574, 2000)

In [212]:
# train_img_paths

In [213]:
%%time 
train_img_data = {}
val_img_data = {}

for path in train_img_paths:
    with open(path,  encoding="utf8", errors='ignore') as f:
        blob = json.load(f)['annotation']
        fname = blob['filename'].strip(".jpg")
#         print(path)
        fname = fname.replace("frame","train")
        no = int(fname.strip("ADE_train_"))
#         print(blob['scene'])
        
        if no>20210:
            continue
        objects = []
        for o in blob['object']:
            objects.append(o['raw_name'])
        
        train_img_data[fname] = {'scene': blob['scene'],
                                 'objects': list(set(objects))
                                }
        
for path in val_img_paths:
    with open(path,  encoding="utf8", errors='ignore') as f:
        blob = json.load(f)['annotation']
        fname = blob['filename'].strip(".jpg")
#         print(path)
        fname = fname.replace("frame","val")
        no = int(fname.strip("ADE_val_"))
#         print(blob['scene'])

        objects = []
        for o in blob['object']:
            objects.append(o['raw_name'])
        
        val_img_data[fname] = {'scene': blob['scene'],
                                 'objects': list(set(objects))
                                }
#         print(fname, no)

len(train_img_data.keys()), len(val_img_data.keys())

CPU times: user 10.7 s, sys: 1.04 s, total: 11.7 s
Wall time: 18.5 s


(20210, 2000)

In [214]:
# train_img_data

In [215]:
np.max([int(key.strip("ADE_train_")) for key in train_img_data.keys()])

20210

In [308]:
train_combined = dict()
val_combined = dict()

coarse_scene_types = {'cultural','home or hotel','industrial','nature landscape','shopping and dining',\
                      'sports and leisure','transportation','unclassified','urban','work place'}
for k, v in train_qa.items():
    ade20k_blob = train_img_data[k] 
    train_combined[k] = {
        'scene-type-coarse':set(ade20k_blob['scene']).intersection(coarse_scene_types),
        'scene-type-fine':set(ade20k_blob['scene']) - coarse_scene_types,
        'caption': train_caps[k],
        'objects':ade20k_blob['objects'],
        'qa-pairs': v,
    }
    

for k, v in val_qa.items():
    ade20k_blob = val_img_data[k] 
    val_combined[k] = {
        'scene-type-coarse':set(ade20k_blob['scene']).intersection(coarse_scene_types),
        'scene-type-fine':set(ade20k_blob['scene']) - coarse_scene_types,
        'caption': val_caps[k],
        'objects':ade20k_blob['objects'],
        'qa-pairs': v,

    }
    

In [309]:
val_combined['ADE_val_00001410']

{'scene-type-coarse': {'urban'},
 'scene-type-fine': {'highway', 'outdoor'},
 'caption': 'In this picture I can see the vehicles on the road. I can see trees on the left side and right side. I can see the light poles. I can see the sky at the top.',
 'objects': ['car', 'traffic light', 'road', 'street light', 'sky', 'trees'],
 'qa-pairs': [['On what side of the road can I see trees?', 'left'],
  ['What do I see on the picture?', 'pole'],
  ['What can I see on the road?', 'vehicle'],
  ['What kind of poles can I see?', 'light'],
  ['What is on the left side of the picture?', 'tree'],
  ['What do I see at the top of the picture?', 'sky'],
  ['What can I see the vehicles on?', 'road']]}

In [218]:
%cd ../../utils/question_generation

/project/dataset-exploration/clip-finetune-ade20k/utils/question_generation


In [219]:
from pipelines import pipeline

qa = pipeline("multitask-qa-qg", model="valhalla/t5-base-qa-qg-hl")

In [192]:
# # for qa pass a dict with "question" and "context"
# nlp({
#     "question": q,
#     "context": c
# })
# q = 'What kind of wood are the card boxes on?'
# q = 'Along with pipes and doors, what is on the top of the card box?'
# c = 'In this image, I see stack of card boxes placed on wooden boards with plastic covers wrapped on them. In the background I see doors, a clock on the wall and few lights and pipes at the top.'

In [261]:
# import nltk
# nltk.download('stopwords')

In [311]:
sw = list(nlp.Defaults.stop_words)
# len(sw)

from nltk.corpus import stopwords
# nltk.download('stopwords')
sw.extend(stopwords.words("english"))

bad_words = ['', '0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 
             'I', 'a', 'above', 'also', 'an', 'and', 'at', 'back', 'background', 'be', 'below',
             'behind', 'beside', 'bottom', 'center', 'can', 'corner', 'different','few', 'five', 'foreground','f',
             'four', 'front', 'group', 'here', 'image', 'is', 'it', 'in','into','item','items',
             'just','kind','left', 'like', 'looks', 'look','many', 'middle', 'number',
             'object', 'objects', 'of', 'one', 'or', 'other', 'on','onto',
             'photo', 'picture', 'placed', 'photograph', 'right', 
             's', 'see', 'seems', 'seen', 'seven', 
             'side', 'six', 'so', 'some', 't', 'taken', 'that', 'the', 'there',
             'them', 'this', 'three', 'they','u',
             'to', 'top', 'two', 'view', 'we', 'which', 'under','zero']

sw.extend(bad_words)

bad_questions = ['what is the name of','what can I see in the picture',
                 'what do I see in the picture','what can we see in the picture',
                 'what do we see in the picture','what can we see in the image',
                 'what do we see in the image','what do I see in the image',
                 'what do you see?','what is this a picture of'
                ]



def validate_qa_pairs(caption, pairs):
    valid_pairs = []
    valid_answers = []
    for pair in pairs:
        q = pair[0]
        
        
        flag=0
        #Filter bad questions
        for bq in bad_questions:
            if q.lower().startswith(bq):
                flag=1
        if flag:
            continue
        
        a = pair[1]
        predicted_answer = qa({
            "question":q,
            "context":caption
        })
        doc = nlp(predicted_answer)
        toks = []
        for t in doc:
            toks.append(t.lemma_)

        
        if a in toks:
            if 1<len(toks)<3:
                predicted_answer = ' '.join([t for t in toks if t not in sw])
                if predicted_answer not in valid_answers:
#                     if len(predicted_answer.split())>1:
#                         print()
#                         print("Q: ", q, "Original: ", a, " | Predicted: ", predicted_answer)
                    valid_pairs.append((q,predicted_answer))
                    valid_answers.append(predicted_answer)
            else:
                if a not in valid_answers:
                    valid_pairs.append((q,a))
                    valid_answers.append(a)
    return valid_pairs

In [303]:
# c = val_combined['ADE_val_00000115']['caption']
# pairs = val_combined['ADE_val_00000115']['qa-pairs']

# validate_qa_pairs(c, pairs)

In [312]:
i=0
print("Validating train pairs")
for k, v in train_combined.items():
    c = v['caption']
    pairs = v['qa-pairs']
    valid_pairs = validate_qa_pairs(c, pairs)
    train_combined[k]['qa-pairs'] = valid_pairs
    if not i%500:
        print(i)
        print(train_combined[k])
    i+=1
        
i=0  
print("Validating val pairs")
for k, v in val_combined.items():
    c = v['caption']
    pairs = v['qa-pairs']
    valid_pairs = validate_qa_pairs(c, pairs)
    val_combined[k]['qa-pairs'] = valid_pairs
    if not i%500:
        print(i)
        print(val_combined[k])
    i+=1

Validating train pairs
0
{'scene-type-coarse': {'home or hotel'}, 'scene-type-fine': {'bedroom', 'indoor'}, 'caption': 'In this picture I can see the inside view of a room, there is a bed with pillows and blankets on it, there are lamps, there is a photo frame attached to the wall, there is a curtain and a window. Through the window I can see water and the sky.', 'objects': ['curtain', 'window', 'wall', 'bed', 'table lamp', 'pillow', 'headboard', 'cushion', 'ceiling', 'picture', 'night table', 'floor', 'frame'], 'qa-pairs': [('What part of the room is a photo frame attached to?', 'wall'), ('Along with a window, what is on the inside of the room?', 'curtain'), ('Along with blankets, what is on the bed?', 'pillow'), ('In this picture I can see the inside view of what?', 'room'), ('What do I see through the window?', 'sky'), ('Where can I see water and the sky?', 'window'), ('What can I see through the window of a room?', 'water'), ('What is attached to the wall of a room?', 'photo'), ('W

4500
{'scene-type-coarse': {'home or hotel'}, 'scene-type-fine': {'indoor', 'hotel_room'}, 'caption': 'In this picture I can see the sofa, bed, mirror on the right side. I can see the wooden table, paper, plant vase in the foreground. I can see the curtain, wooden object, electronic device on the left side. I can see the hanging light.', 'objects': ['window', 'magazine', 'wall', 'bed', 'vase', 'dresser', 'curtain', 'skirt', 'back', 'flowers', 'ceiling', 'coffee table', 'floor', 'sofa', 'seat base', 'seat cushion', 'mirror', 'arm', 'semi-flush mount light', 'radiator'], 'qa-pairs': [('What is on the right side of the picture?', 'sofa'), ('On which side of the picture is the sofa?', 'right'), ('What type of device is on the left side of the picture?', 'electronic'), ('What type of table is in the foreground?', 'wooden'), ('What type of vase is in the foreground?', 'plant vase')]}
5000
{'scene-type-coarse': {'work place'}, 'scene-type-fine': {'parking_garage indoor', 'indoor'}, 'caption':

10000
{'scene-type-coarse': {'home or hotel'}, 'scene-type-fine': {'indoor', 'dining_room'}, 'caption': 'There are chairs around the table and a bowl on the table in the foreground area of the image, there are windows, curtains, lamp frame, greenery and the sky in the background and a chandelier at the top side.', 'objects': ['window', 'curtain', 'wall', 'bowl', 'table lamp', 'chandelier', 'ceiling', 'picture', 'armchair', 'switch', 'table', 'floor', 'shelves', 'chair', 'cabinet'], 'qa-pairs': [('What is on the top of the image?', 'chandelier'), ('What is in the foreground area of the image?', 'bowl'), ('What is on the frame of the image?', 'lamp frame')]}
10500
{'scene-type-coarse': {'work place'}, 'scene-type-fine': {'indoor', 'maternity_ward'}, 'caption': 'There are cribs at the bottom of this image. There is one person standing and wearing a white color dress. We can see some objects and a window on the left side of this image. There is a wall in the background.', 'objects': ['wind

16500
{'scene-type-coarse': {'home or hotel'}, 'scene-type-fine': {'living_room', 'indoor'}, 'caption': 'In this image on the left side, we can see the sofas. And we can see the lamp and cupboard. And we can see the frames on the wall. And on the right, we can see the monitor. And we can see some objects which are arranged in the wooden cupboard.', 'objects': ['wall', 'books', 'cushion', 'picture', 'shelf', 'pane', 'back pillow', 'oven', 'side', 'television', 'top', 'sofa', 'floor', 'ottoman', 'handle', 'work surface', 'desk lamp', 'ceiling', 'loudspeaker', 'door', 'seat cushion', 'frame', 'arm', 'leg', 'music system', 'front', 'shelves', 'cabinet', 'ceiling recessed light'], 'qa-pairs': [('On what part of the house can we see the frames?', 'wall'), ('In the image on the left, we can see the lamp and what else?', 'cupboard'), ('Along with the cupboard, what is visible in the image on the left?', 'lamp'), ('On which side of the image is the monitor?', 'right'), ('What kind of cupboard i

500
{'scene-type-coarse': {'urban'}, 'scene-type-fine': {'outdoor', 'street'}, 'caption': 'In this picture we can see houses beside the road, where we can see shops, there is text on top of shop, there is black gate, there are glass windows to the houses.', 'objects': ['window', 'alarm', 'road', 'shop window', 'building', 'mailbox', 'sidewalk', 'roof', 'door', 'lock', 'handle', 'brand name'], 'qa-pairs': [('What is on top of the shop?', 'text'), ('What color is the gate in the picture?', 'black'), ('What kind of windows are on the houses?', 'glass'), ('What can we see beside the road?', 'house'), ('Where can we see shops?', 'road')]}
1000
{'scene-type-coarse': {'urban'}, 'scene-type-fine': {'yard', 'outdoor'}, 'caption': 'In the picture I can see flower plants and the grass. In the background I can see trees and the sky.', 'objects': ['sky', 'grass', 'plants', 'plant'], 'qa-pairs': [('Along with the grass, what type of plants can be seen in the picture?', 'flower plant'), ('Along with 

In [328]:
# for k, v in train_combined.items():
#     train_combined[k]['scene-type-coarse'] = train_combined[k]['scene-type-coarse'].pop()
#     train_combined[k]['scene-type-fine'] = list(train_combined[k]['scene-type-fine'])

# for k, v in val_combined.items():
#     val_combined[k]['scene-type-coarse'] = val_combined[k]['scene-type-coarse'].pop()
#     val_combined[k]['scene-type-fine'] = list(val_combined[k]['scene-type-fine'])

# train_combined

In [316]:
%cd ../../data/

/project/dataset-exploration/clip-finetune-ade20k/data


In [330]:
# with jsonlines.open('ade20k-train-combined.jsonl','w') as f:
#     for k, v in train_combined.items():
#         f.write({k:v})
        
# with jsonlines.open('ade20k-val-combined.jsonl','w') as f:
#     for k, v in val_combined.items():
#         f.write({k:v})

In [480]:
import collections 
import nltk  # $ pip install nltk
from nltk.corpus import cmudict  # >>> nltk.download('cmudict')

all_objects = []

for k, v in train_combined.items():
    all_objects.extend(v['objects'])
    
# len(all_objects), len(set(all_objects))

acceptable = []
for k,v in collections.Counter(all_objects).most_common():
#     print(k)
#     print(v)
#     break
    if v>=10:
        acceptable.append(k)
        
all_objects = set(acceptable)
len(all_objects)

904

In [481]:
def starts_with_vowel_sound(word, pronunciations=cmudict.dict()):
    for syllables in pronunciations.get(word, []):
        return syllables[0][-1].isdigit()  # use only the first one

x = []

for obj in all_objects:
    for i, doc in enumerate(nlp(obj)):
        if i==0:
            if doc.tag_ != 'NNS':
                article = "an "+ obj if starts_with_vowel_sound(doc.text) else "a " + obj
            else:
                article = None
            x.append((obj, article))

all_objects = dict()

for i in x:
    all_objects[i[0]] = i[1]

In [487]:
question_templates_singular = ["Is there %s?", "Is there %s in this image?"]
question_templates_plural = ["Are there %s?", "Are there %s in this photo?", "Are there %s in this image?"]
question_templates_generic = ["Do you see %s?", "Can you see %s?", "Do you see %s in this picture?",
                              "Can you see %s in this photo?", "Does this picture have %s?",
                              "In this image, can you see %s?", "Can we see %s in this picture?",
                              "Does this image consist of %s?", "Does this photo have %s?",
                             ]

question_templates_singular.extend(question_templates_generic)
question_templates_plural.extend(question_templates_generic)

import random


def create_binary_questions(objects):
    objects = set(objects)
    all_objects_ = all_objects.keys()
    
    yes = objects.intersection(all_objects_)
    no = all_objects_ - objects
    
    question_pairs = []
    
    for item in yes:
        if all_objects[item]:
            string = random.choice(question_templates_singular) % all_objects[item]
            question_pairs.append((string, "yes"))
        else:
            string = random.choice(question_templates_plural) % item
            question_pairs.append((string, "yes"))
            
    no_sample_size = max(2, len(yes) + random.choice(range(-3,3)))
    no = random.sample(list(no), no_sample_size)
    
    for item in no:
        if all_objects[item]:
            string = random.choice(question_templates_singular) % all_objects[item]
            question_pairs.append((string, "no"))
        else:
            string = random.choice(question_templates_plural) % item
            question_pairs.append((string, "no"))
            
    return question_pairs

In [488]:
for k, v in train_combined.items():
    train_combined[k]['qa-binary'] = create_binary_questions(train_combined[k]['objects'])

for k, v in val_combined.items():
    val_combined[k]['qa-binary'] = create_binary_questions(val_combined[k]['objects'])

In [491]:
with jsonlines.open('ade20k-train-combined.jsonl','w') as f:
    for k, v in train_combined.items():
        f.write({k:v})
        
with jsonlines.open('ade20k-val-combined.jsonl','w') as f:
    for k, v in val_combined.items():
        f.write({k:v})

### creating vqa train and val sets

In [38]:
import jsonlines
train = dict()
val = dict()

basepath = "../data/"

with jsonlines.open(basepath+'ade20k-train-combined.jsonl','r') as f:
    for row in f:
        train.update(row)        

with jsonlines.open(basepath+'ade20k-val-combined.jsonl','r') as f:
    for row in f:
        val.update(row)        
len(train.keys()), len(val.keys())

(20210, 2000)

In [39]:
img_train_qa_triplets = []
img_val_qa_triplets = []

for k, v in train.items():
    triplets = []
    for pair in v['qa-pairs']:
        triplets.append([k, pair[0], pair[1]])
    for pair in v['qa-binary']:
        triplets.append([k, pair[0], pair[1]])
    img_train_qa_triplets.extend(triplets)
    
for k, v in val.items():
    triplets = []
    for pair in v['qa-pairs']:
        triplets.append([k, pair[0], pair[1]])
    for pair in v['qa-binary']:
        triplets.append([k, pair[0], pair[1]])
    img_val_qa_triplets.extend(triplets)


len(img_train_qa_triplets), len(img_val_qa_triplets)

(630237, 69461)

In [40]:
import pandas as pd

train_df = pd.DataFrame(img_train_qa_triplets, columns=['img_id','question','answer'])
val_df = pd.DataFrame(img_val_qa_triplets, columns=['img_id','question','answer'])

print(train_df.shape, val_df.shape)

(630237, 3) (69461, 3)


In [41]:
vqa_train_labels = set(train_df.answer.to_list())
vqa_val_labels = set(val_df.answer.to_list())

intersecting_labels = vqa_train_labels.intersection(vqa_val_labels)
intersecting_labels = set([label for label in intersecting_labels if len(label)>0])

len(intersecting_labels)

1025

In [42]:
train_df = train_df[train_df.answer.isin(intersecting_labels)]
val_df = val_df[val_df.answer.isin(intersecting_labels)]

train_df.shape, val_df.shape

((622557, 3), (69079, 3))

In [43]:
import collections

c = collections.Counter(train_df.answer)
c = collections.Counter({k: c for k, c in c.items() if c >= 5})

valid_labels = c.keys()
'yes' in valid_labels, 'dadas' in valid_labels

(True, False)

In [44]:
train_df = train_df[train_df.answer.isin(valid_labels)]
val_df = val_df[val_df.answer.isin(valid_labels)]

train_df.shape, val_df.shape

((621960, 3), (68761, 3))

In [47]:
train_df.to_csv("../data/ADE20K-QA/vqa_train_df.tsv", sep="\t",index=False)
val_df.to_csv("../data/ADE20K-QA/vqa_val_df.tsv", sep="\t",index=False)

In [48]:
train_df

,img_id,question,answer
0,ADE_train_00003661,What part of the room is a photo frame attache...,wall
1,ADE_train_00003661,"Along with a window, what is on the inside of ...",curtain
2,ADE_train_00003661,"Along with blankets, what is on the bed?",pillow
3,ADE_train_00003661,In this picture I can see the inside view of w...,room
4,ADE_train_00003661,What do I see through the window?,sky
...,...,...,...
630232,ADE_train_00016390,"In this image, can you see a drawer?",no
630233,ADE_train_00016390,Can you see candies?,no
630234,ADE_train_00016390,Does this picture have a telephone?,no
630235,ADE_train_00016390,Is there a pulpit?,no


In [55]:
img_train_caption_pairs = []
img_val_caption_pairs = []

for k, v in train.items():
    caption = v['caption']
    img_train_caption_pairs.append((k, caption))
    

for k, v in val.items():
    caption = v['caption']
    img_val_caption_pairs.append((k, caption))

len(img_train_caption_pairs), len(img_val_caption_pairs)

(20210, 2000)

In [58]:
caption_train = pd.DataFrame(img_train_caption_pairs, columns=['img_id','caption'])
caption_val = pd.DataFrame(img_val_caption_pairs, columns=['img_id','caption'])

caption_train.shape, caption_val.shape

((20210, 2), (2000, 2))

In [60]:
caption_train.to_csv("../data/ADE20K-QA/caption_train_df.tsv", sep="\t",index=False)
caption_val.to_csv("../data/ADE20K-QA/caption_val_df.tsv", sep="\t",index=False)

In [62]:
pd.read_csv("../data/ADE20K-QA/caption_val_df.tsv", sep="\t")

,img_id,caption
0,ADE_val_00001410,In this picture I can see the vehicles on the ...
1,ADE_val_00001113,In this image I can see water and I can also s...
2,ADE_val_00000424,"In this image we can see a table, light, book,..."
3,ADE_val_00001210,This image is taken outdoors. At the top of th...
4,ADE_val_00001480,"In this image, there are cupboards, stove, mic..."
...,...,...
1995,ADE_val_00000922,In the image there is a wall and there are two...
1996,ADE_val_00001449,"In the foreground, I can see a fire hydrant, a..."
1997,ADE_val_00001098,In this picture I can see a toilet on the floo...
1998,ADE_val_00001735,In the image I can see the the view of a place...
